In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Conv3D, Conv2D
import yaml


In [2]:
with open('configs/config_file.yaml') as file:
    params = yaml.safe_load(file)

print(params)

window_size = params['training']['window_size']
h_step = params['forecast']['h_step']
patience = params['training']['patience']
epsilon = params['training']['epsilon']
batch_size = params['training']['batch_size']
epochs = params['training']['epochs']

run = params['model']['run']
learning_rate = params['model']['lr']
covariate_columns = params['model']['covariates']
option_type = params['model']['option']

{'training': {'window_size': 21, 'batch_size': 32, 'patience': 15, 'epsilon': 1e-06, 'random_state': 42, 'epochs': 100, 'covariates': 'None'}, 'model': {'run': 'short_ttm', 'option': 'put', 'filters': 2, 'kernel_size': [2, 2], 'strides': 1, 'kernel_initializer': 'glorot_uniform', 'recurrent_initializer': 'orthogonal', 'optimizer': 'adam', 'lr': 0.001, 'covariates': []}, 'forecast': {'h_step': 1}}


In [3]:
# Let's reshape our input data of the thing... we are going to need labels, and we are going to need train surface.
# The labels will be, the smoothed IVs of our data
# The train will be the dimensions, with time x ttm x moneyness encoders
# If we have covariates, the channels will be larger? -> Yes, starting channels will be added to the layers

# Load the data first
if run == 'short_ttm':
    data_train = pd.read_csv('data/final/smoothed/data_train.csv')
    data_val = pd.read_csv('data/final/evaluation/validation_set.csv')
    data_test = pd.read_csv('data/final/evaluation/test_set.csv')

    if covariate_columns is not None:
        covar_df = pd.read_excel('data/final/covariates/covariates_train.xlsx')
        covar_df_val = pd.read_excel('data/final/covariates/covariates_validation.xlsx')

        covar_df = covar_df.rename(columns={'Date':'date'})
        covar_df_val = covar_df_val.rename(columns={'Date':'date'})
        covar_df = covar_df[['date'] + covariate_columns]
        covar_df_val = covar_df_val[['date'] + covariate_columns]

elif run == 'long_ttm':
    data_train = pd.read_csv('data/final/smoothed/data_train_long.csv')
    data_val = pd.read_csv('data/final/evaluation/validation_set_long.csv')
    data_test = pd.read_csv('data/final/evaluation/test_set_long.csv')

    if covariate_columns is not None:
        covar_df = pd.read_excel('data/final/covariates/covariates_train_long.xlsx')
        covar_df_val = pd.read_excel('data/final/covariates/covariates_validation_long.xlsx')

        covar_df = covar_df.rename(columns={'Date':'date'})
        covar_df_val = covar_df_val.rename(columns={'Date':'date'})
        covar_df = covar_df[['date'] + covariate_columns]
        covar_df_val = covar_df_val[['date'] + covariate_columns]
   
else:
    print('Select a dataset')


In [4]:
print(covar_df.isna().sum())
print(covar_df)
covar_df = covar_df.sort_values('date').ffill()
print(covar_df.isna().sum())

date    0
dtype: int64
           date
0    2012-01-03
1    2012-01-04
2    2012-01-05
3    2012-01-06
4    2012-01-09
...         ...
2709 2023-02-22
2710 2023-02-23
2711 2023-02-24
2712 2023-02-27
2713 2023-02-28

[2714 rows x 1 columns]
date    0
dtype: int64


In [5]:
data_train['date'] = pd.to_datetime(data_train['date'])
data_val['date'] = pd.to_datetime(data_val['date'])
data_test['date'] = pd.to_datetime(data_test['date'])
data_train = pd.merge(data_train, covar_df, on='date', how='left')
data_val = pd.merge(data_val, covar_df, on='date', how='left')
data_test = pd.merge(data_test, covar_df, on='date', how='left')

In [6]:
pd.set_option('display.max_row', None)

In [7]:
def process(data, eval=False, option_type='both'):
    data = data.drop(columns="Unnamed: 0")

    if option_type =='put':
        data = data[data['cp_flag']=='P']
    elif option_type =='call':
        data = data[data['cp_flag']=='C']
    # Let's implement the thing, where deep OTM, OTM, ATM, ITM, deep ITM is a thing

    # we have to discriminate between calls and puts
    # Coding; deep OTM = 1, OTM =2, ATM =3, ITM = 4, deep ITM=5 
    # outliers, sort of?
    print(data.shape)
    data = data[data['moneyness'] >= 0.8]
    data = data[data['moneyness'] <= 1.6]
    print(data.shape)

    # Also consider what to do with low volume... probably include them and acknowledge them as a limitation

    data.loc[(data['cp_flag']=='C') & (data['moneyness'] <0.90), 'moneyness_enc'] = 1
    data.loc[(data['cp_flag']=='C') & (data['moneyness'] >=0.90) & (data['moneyness'] < 0.97), 'moneyness_enc'] = 2
    data.loc[(data['cp_flag']=='C') & (data['moneyness'] >=0.97) & (data['moneyness'] < 1.03), 'moneyness_enc'] = 3
    data.loc[(data['cp_flag']=='C') & (data['moneyness'] >=1.03) & (data['moneyness'] < 1.10), 'moneyness_enc'] = 4
    data.loc[(data['cp_flag']=='C') & (data['moneyness'] >=1.10), 'moneyness_enc'] = 5

    data.loc[(data['cp_flag']=='P') & (data['moneyness'] <0.90), 'moneyness_enc'] = 5
    data.loc[(data['cp_flag']=='P') & (data['moneyness'] >=0.90) & (data['moneyness'] < 0.97), 'moneyness_enc'] = 4
    data.loc[(data['cp_flag']=='P') & (data['moneyness'] >=0.97) & (data['moneyness'] < 1.03), 'moneyness_enc'] = 3
    data.loc[(data['cp_flag']=='P') & (data['moneyness'] >=1.03) & (data['moneyness'] < 1.10), 'moneyness_enc'] = 2
    data.loc[(data['cp_flag']=='P') & (data['moneyness'] >=1.10), 'moneyness_enc'] = 1

    # multiple values of the same moneyness and maturity encoding, if that's the case, we take the average
    if eval:
        # data_avg = data.groupby(['date', 'moneyness_enc', 'maturity'], as_index=False)['impl_volatility'].mean()
        # print(data.shape)
        # print(data.groupby(['date','maturity', 'moneyness_enc']).size())
        # data = data.drop_duplicates(subset=['date', 'moneyness_enc', 'maturity'])
        # print(data.shape)
        # data = data.drop(columns='impl_volatility')

        # data = pd.merge(data, data_avg, on=['date','moneyness_enc', 'maturity'], how='left')
        print('eval')
    else:
        data_avg = data.groupby(['date', 'moneyness_enc', 'maturity'], as_index=False)['IV_smooth'].mean()
        print(data.shape)
        print(data.groupby(['date', 'maturity', 'moneyness_enc']).size())
        data = data.drop_duplicates(subset=['date', 'moneyness_enc', 'maturity'])
        print(data.shape)
        data = data.drop(columns='IV_smooth')

        data = pd.merge(data, data_avg, on=['date','moneyness_enc', 'maturity'], how='left')

    # data = data.drop_duplicates(subset=['date', 'moneyness_enc', 'maturity'])
    print(data.shape)
    return data

# data_train = process(data_train, False, option_type=option_type)
# data_val = process(data_val, eval=True, option_type=option_type)
# data_test = process(data_test, eval=True, option_type=option_type)

# Thing to fix: The moneyness encoded, results in multiple impl volatility values for the same moneyness, maturity
# combination. To fix this, take the average, and omit the others

In [8]:
def process(data, eval=False, option_type='both'):
    data = data.drop(columns="Unnamed: 0")

    if option_type =='put':
        data = data[data['cp_flag']=='P']
    elif option_type =='call':
        data = data[data['cp_flag']=='C']
    # Let's implement the thing, where deep OTM, OTM, ATM, ITM, deep ITM is a thing

    # we have to discriminate between calls and puts
    # Coding; deep OTM = 1, OTM =2, ATM =3, ITM = 4, deep ITM=5 
    # outliers, sort of?
    print(data.shape)
    data = data[data['moneyness'] >= 0.8]
    data = data[data['moneyness'] <= 1.6]
    print(data.shape)

    # Also consider what to do with low volume... probably include them and acknowledge them as a limitation

    data.loc[(data['cp_flag']=='C') & (data['moneyness'] <0.85), 'moneyness_enc'] = 1
    data.loc[(data['cp_flag']=='C') & (data['moneyness'] >=0.85) & (data['moneyness'] < 0.90), 'moneyness_enc'] = 2
    data.loc[(data['cp_flag']=='C') & (data['moneyness'] >=0.90) & (data['moneyness'] < 0.95), 'moneyness_enc'] = 3
    data.loc[(data['cp_flag']=='C') & (data['moneyness'] >=0.95) & (data['moneyness'] < 1.00), 'moneyness_enc'] = 4
    data.loc[(data['cp_flag']=='C') & (data['moneyness'] >=1.00) & (data['moneyness'] < 1.05), 'moneyness_enc'] = 5
    data.loc[(data['cp_flag']=='C') & (data['moneyness'] >=1.05) & (data['moneyness'] < 1.10), 'moneyness_enc'] = 6
    data.loc[(data['cp_flag']=='C') & (data['moneyness'] >=1.10) & (data['moneyness'] < 1.15), 'moneyness_enc'] = 7
    data.loc[(data['cp_flag']=='C') & (data['moneyness'] >=1.15), 'moneyness_enc'] = 8

    data.loc[(data['cp_flag']=='P') & (data['moneyness'] <0.85), 'moneyness_enc'] = 8
    data.loc[(data['cp_flag']=='P') & (data['moneyness'] >=0.85) & (data['moneyness'] < 0.90), 'moneyness_enc'] = 7
    data.loc[(data['cp_flag']=='P') & (data['moneyness'] >=0.90) & (data['moneyness'] < 0.95), 'moneyness_enc'] = 6
    data.loc[(data['cp_flag']=='P') & (data['moneyness'] >=0.95) & (data['moneyness'] < 1.00), 'moneyness_enc'] = 5
    data.loc[(data['cp_flag']=='P') & (data['moneyness'] >=1.00) & (data['moneyness'] < 1.05), 'moneyness_enc'] = 4
    data.loc[(data['cp_flag']=='P') & (data['moneyness'] >=1.05) & (data['moneyness'] < 1.10), 'moneyness_enc'] = 3
    data.loc[(data['cp_flag']=='P') & (data['moneyness'] >=1.10) & (data['moneyness'] < 1.15), 'moneyness_enc'] = 2
    data.loc[(data['cp_flag']=='P') & (data['moneyness'] >=1.15), 'moneyness_enc'] = 1
    # multiple values of the same moneyness and maturity encoding, if that's the case, we take the average
    if eval:
        # data_avg = data.groupby(['date', 'moneyness_enc', 'maturity'], as_index=False)['impl_volatility'].mean()
        # print(data.shape)
        # print(data.groupby(['date','maturity', 'moneyness_enc']).size())
        # data = data.drop_duplicates(subset=['date', 'moneyness_enc', 'maturity'])
        # print(data.shape)
        # data = data.drop(columns='impl_volatility')

        # data = pd.merge(data, data_avg, on=['date','moneyness_enc', 'maturity'], how='left')
        print('eval')
    else:
        data_avg = data.groupby(['date', 'moneyness_enc', 'maturity'], as_index=False)['IV_smooth'].mean()
        print(data.shape)
        print(data.groupby(['date', 'maturity', 'moneyness_enc']).size())
        data = data.drop_duplicates(subset=['date', 'moneyness_enc', 'maturity'])
        print(data.shape)
        data = data.drop(columns='IV_smooth')

        data = pd.merge(data, data_avg, on=['date','moneyness_enc', 'maturity'], how='left')

    # data = data.drop_duplicates(subset=['date', 'moneyness_enc', 'maturity'])
    print(data.shape)
    return data

data_train = process(data_train, False, option_type=option_type)
data_val = process(data_val, eval=True, option_type=option_type)
data_test = process(data_test, eval=True, option_type=option_type)

# Thing to fix: The moneyness encoded, results in multiple impl volatility values for the same moneyness, maturity
# combination. To fix this, take the average, and omit the others

(356419, 29)
(355364, 29)
(355364, 30)
date        maturity  moneyness_enc
2012-01-03  3         3.0                5
                      4.0               12
                      5.0                9
2012-01-04  2         3.0                2
                      4.0               12
                      5.0                4
2012-01-05  1         4.0               11
                      5.0                5
2012-01-06  5         2.0                3
                      3.0               11
                      4.0               12
                      5.0                7
2012-01-09  4         3.0                4
                      4.0               13
                      5.0                6
2012-01-10  3         3.0                5
                      4.0               12
                      5.0                9
                      6.0                1
2012-01-11  2         3.0                1
                      4.0               12
                      

In [9]:
# def frame_to_numpy(data, eval=False):
#     # Convert 'time_step' to datetime
#     data['time_step'] = pd.to_datetime(data['date'])

#     # Create a time_step index (e.g., from the first unique date)
#     time_step_index = pd.to_datetime(data['time_step']).dt.strftime('%Y-%m-%d').unique()

#     # Map time_step dates to integer index
#     data['time_step_idx'] = data['time_step'].apply(lambda x: np.where(time_step_index == x.strftime('%Y-%m-%d'))[0][0])
#     print(data['time_step_idx'])

#     maturity_values = np.sort(data['maturity'].unique())
#     maturity_to_idx = {mat: i for i, mat in enumerate(maturity_values)}

#     time_steps = len(time_step_index)
#     money_dim = len(data['moneyness_enc'].unique())
#     ttm_dim = len(data['maturity'].unique())

#     # Create an empty numpy array with the shape (time_steps, height_dim, width_dim)
#     IV_array = np.zeros((time_steps, money_dim, ttm_dim, ), dtype=np.float32)

#     # Populate the numpy array with values from the DataFrame
#     for idx, row in data.iterrows():
#         time_step_idx = row['time_step_idx']
#         height = int(row['moneyness_enc']) - 1 
#         width = maturity_to_idx[row['maturity']]
        
#         if eval==False:
#             value = row['IV_smooth']
#         else:
#             value = row['impl_volatility']
            
#         # print(time_step_idx, height, width, value)
#         # Assign the value to the corresponding position in the numpy array
#         IV_array[time_step_idx, height, width] = value
        
#     IV_array = IV_array.reshape((IV_array.shape[0], money_dim, ttm_dim, 1))
#     return IV_array

In [10]:
# def frame_to_numpy(data, covariate_cols=None, eval=False):
    
#     data['time_step'] = pd.to_datetime(data['date'])
#     time_step_index = pd.to_datetime(data['time_step']).dt.strftime('%Y-%m-%d').unique()
#     data['time_step_idx'] = data['time_step'].apply(lambda x: np.where(time_step_index == x.strftime('%Y-%m-%d'))[0][0])

#     maturity_values = np.sort(data['maturity'].unique())
#     maturity_to_idx = {mat: i for i, mat in enumerate(maturity_values)}

#     time_steps = len(time_step_index)
#     money_dim = len(data['moneyness_enc'].unique())
#     ttm_dim = len(maturity_values)

#     # Base IV tensor
#     IV_array = np.zeros((time_steps, money_dim, ttm_dim))

#     # If covariates provided, create tensor to hold them
#     covariate_arrays = {}
#     if covariate_cols:
#         for cov in covariate_cols:
#             covariate_arrays[cov] = np.zeros((time_steps, money_dim, ttm_dim), dtype=np.float32)

#     for idx, row in data.iterrows():
#         time_step_idx = row['time_step_idx']
#         height = int(row['moneyness_enc']) - 1 
#         width = maturity_to_idx[row['maturity']]
#         value = row['IV_smooth'] if not eval else row['impl_volatility']
#         IV_array[time_step_idx, height, width] = value

#         # Also fill in covariates
#         if covariate_cols:
#             for cov in covariate_cols:
#                 covariate_arrays[cov][time_step_idx, height, width] = row[cov]

#     # Reshape and concatenate
#     IV_array = IV_array.reshape((time_steps, money_dim, ttm_dim, 1))

#     if covariate_cols:
#         covariate_stack = [arr.reshape((time_steps, money_dim, ttm_dim, 1)) for arr in covariate_arrays.values()]
#         covariate_stack = np.concatenate(covariate_stack, axis=-1)  # shape: (T, H, W, C)
#         IV_array = np.concatenate([IV_array, covariate_stack], axis=-1)  # final shape: (T, H, W, 1+C)

#     return IV_array


In [11]:
pd.set_option('display.max_row', 10)
print(data_train)

            date               symbol      exdate   last_date cp_flag  \
0     2012-01-03  SPXW 120106P1195000  2012-01-06  03/01/2012       P   
1     2012-01-03  SPXW 120106P1220000  2012-01-06  03/01/2012       P   
2     2012-01-03  SPXW 120106P1280000  2012-01-06  03/01/2012       P   
3     2012-01-04  SPXW 120106P1210000  2012-01-06  04/01/2012       P   
4     2012-01-04  SPXW 120106P1220000  2012-01-06  04/01/2012       P   
...          ...                  ...         ...         ...     ...   
21257 2021-12-06  SPXW 211213P4000000  2021-12-13  06/12/2021       P   
21258 2021-12-06  SPXW 211213P4175000  2021-12-13  06/12/2021       P   
21259 2021-12-06  SPXW 211213P4375000  2021-12-13  06/12/2021       P   
21260 2021-12-06  SPXW 211213P4595000  2021-12-13  06/12/2021       P   
21261 2021-12-06  SPXW 211213P4840000  2021-12-13  06/12/2021       P   

       strike_price  best_bid  best_offer  volume  open_interest  ...  \
0              1195      0.10        0.20     479 

In [12]:
def frame_to_numpy(data, covariate_cols=None, eval=False):
    
    data['time_step'] = data['date']
    time_step_index = pd.to_datetime(data['time_step']).dt.strftime('%Y-%m-%d').unique()
    date_to_index = {date: idx for idx, date in enumerate(time_step_index)}

    data['time_step_str'] = data['time_step'].dt.strftime('%Y-%m-%d')
    data['time_step_idx'] = data['time_step_str'].map(date_to_index)
    #data['time_step_idx'] = data['time_step'].apply(lambda x: np.where(time_step_index == x.strftime('%Y-%m-%d'))[0][0])

    maturity_values = np.sort(data['maturity'].unique())
    maturity_to_idx = {mat: i for i, mat in enumerate(maturity_values)}

    time_steps = len(time_step_index)
    money_dim = len(data['moneyness_enc'].unique())
    ttm_dim = len(maturity_values)

    # Base IV tensor
    IV_array = np.zeros((time_steps, money_dim, ttm_dim, 1+len(covariate_cols)))

    for idx, row in data.iterrows():
        time_step_idx = row['time_step_idx']
        height = int(row['moneyness_enc']) - 1 
        width = maturity_to_idx[row['maturity']]
        value = row['IV_smooth'] if not eval else row['impl_volatility']
        IV_array[time_step_idx, height, width, 0] = value

        for i, cov in enumerate(covariate_cols):
            IV_array[time_step_idx, :, :, 1+i] = row[cov]

    return IV_array


In [13]:
IV_train = frame_to_numpy(data_train, covariate_columns)
IV_val = frame_to_numpy(data_val, covariate_columns, eval=True)
IV_test = frame_to_numpy(data_test, covariate_columns, eval=True)

In [14]:
# IV_train = frame_to_numpy(data_train)
# IV_val = frame_to_numpy(data_val, eval=True)
# IV_test = frame_to_numpy(data_test, eval=True)
# 7 min on pc, for long set
# Write array to data folder?

In [15]:
print(IV_train[2001])

[[[0.        ]
  [0.        ]
  [0.        ]
  [0.52302127]
  [0.        ]]

 [[0.        ]
  [0.41650023]
  [0.        ]
  [0.43218356]
  [0.        ]]

 [[0.40849711]
  [0.33387112]
  [0.        ]
  [0.36716021]
  [0.        ]]

 [[0.35231177]
  [0.26866499]
  [0.        ]
  [0.2951108 ]
  [0.        ]]

 [[0.23871904]
  [0.1915956 ]
  [0.        ]
  [0.22431286]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]]


In [16]:
print(IV_train[2001])

[[[0.        ]
  [0.        ]
  [0.        ]
  [0.52302127]
  [0.        ]]

 [[0.        ]
  [0.41650023]
  [0.        ]
  [0.43218356]
  [0.        ]]

 [[0.40849711]
  [0.33387112]
  [0.        ]
  [0.36716021]
  [0.        ]]

 [[0.35231177]
  [0.26866499]
  [0.        ]
  [0.2951108 ]
  [0.        ]]

 [[0.23871904]
  [0.1915956 ]
  [0.        ]
  [0.22431286]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]]


In [17]:
print(IV_train[2001][0][2][0:1])

[0.]


In [18]:
print(IV_train[2001][0][2][0:1])

[0.]


In [19]:
print(IV_train.shape, IV_val.shape, IV_test.shape)

(2406, 8, 5, 1) (114, 8, 5, 1) (194, 8, 5, 1)


In [20]:
# # Convert 'time_step' to datetime
# data_train['time_step'] = pd.to_datetime(data_train['date'])

# # Create a time_step index (e.g., from the first unique date)
# time_step_index = pd.to_datetime(data_train['time_step']).dt.strftime('%Y-%m-%d').unique()

# # Map time_step dates to integer index
# data_train['time_step_idx'] = data_train['time_step'].apply(lambda x: np.where(time_step_index == x.strftime('%Y-%m-%d'))[0][0])
# print(data_train['time_step_idx'])
# ttm_dim = 5
# money_dim = 5
# time_steps = len(time_step_index)

# # Create an empty numpy array with the shape (time_steps, height_dim, width_dim)
# IV_array = np.zeros((time_steps, ttm_dim, money_dim))

# # Populate the numpy array with values from the DataFrame
# for idx, row in data_train.iterrows():
#     time_step_idx = row['time_step_idx']
#     width = row['maturity'] - 1 
#     height = int(row['moneyness_enc']) - 1 
#     value = row['IV_smooth']
#     print(time_step_idx, width, height, value)
#     # Assign the value to the corresponding position in the numpy array
#     IV_array[time_step_idx, height, width] = value

In [21]:
dataset_train = tf.keras.utils.timeseries_dataset_from_array(
    data=IV_train,
    targets=IV_train[window_size:][:,:,:,0:1], # Select only the IV, not the covariates
    sequence_length=window_size,
    batch_size=batch_size
)

In [22]:
# window_size = 21 # about one month
# labels = IV_array[1:]
# X = IV_array[:-1]

# dataset_train = tf.keras.utils.timeseries_dataset_from_array(
#     data=IV_train[:-1],
#     targets=IV_train[window_size:],
#     sequence_length=window_size,
#     batch_size=batch_size
# )

# Add the last timepoints of the dataset to the validation set, for the computation of the
# validation set performance is calculated within the window size too
# So the validation set should start from the end of the training set

IV_val_input = np.concatenate((IV_train[-window_size:], IV_val), axis=0)

dataset_val = tf.keras.utils.timeseries_dataset_from_array(
    data=IV_val_input,
    targets=IV_val_input[window_size:][:,:,:,0:1],
    sequence_length=window_size,
    batch_size=batch_size
)

IV_test_input = np.concatenate((IV_val[-window_size:], IV_test), axis=0)

dataset_test= tf.keras.utils.timeseries_dataset_from_array(
    data=IV_test_input,
    targets=IV_test_input[window_size:][:,:,:,0:1],
    sequence_length=window_size,
    batch_size=batch_size
)

In [23]:
print(dataset_train)

<BatchDataset element_spec=(TensorSpec(shape=(None, None, 8, 5, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, 8, 5, 1), dtype=tf.float64, name=None))>


In [122]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, ConvLSTM2D, BatchNormalization, TimeDistributed
# from tensorflow.keras.layers import Dense, Flatten, Reshape, Concatenate, Conv2D
# from tensorflow.keras.optimizers import Adam


In [123]:
# # Define constants
# TIME_STEPS = window_size
# HEIGHT = len(data_train['moneyness_enc'].unique())
# WIDTH = len(data_train['maturity'].unique())
# CHANNELS = 1
# COVARIATE_DIM = len(covariate_columns)

# # IV input: (batch, time, height, width, 1)
# iv_input = Input(shape=(TIME_STEPS, HEIGHT, WIDTH, CHANNELS), name='iv_input')

# # Covariates input: (batch, time, num_covariates)
# cov_input = Input(shape=(TIME_STEPS, COVARIATE_DIM), name='cov_input')


In [124]:
# # Flatten covariates across time
# x_cov = TimeDistributed(Dense(32, activation='relu'))(cov_input)  # Shape: (batch, time, 32)
# x_cov = TimeDistributed(Dense(HEIGHT * WIDTH, activation='relu'))(x_cov)
# x_cov = Reshape((TIME_STEPS, HEIGHT, WIDTH, 1))(x_cov)  # Fake a spatial dimension


In [125]:
# # Merge IV and covariate paths
# merged = Concatenate(axis=-1)([iv_input, x_cov])  # Shape: (batch, time, height, width, 2)


In [126]:
# x = ConvLSTM2D(filters=64, kernel_size=(3, 3),
#                padding='same', return_sequences=True)(merged)
# x = BatchNormalization()(x)

# x = ConvLSTM2D(filters=64, kernel_size=(3, 3),
#                padding='same', return_sequences=False)(x)
# x = BatchNormalization()(x)

# output = Conv2D(filters=1, kernel_size=(1, 1),
#                 activation='relu', padding='same')(x)


In [127]:
# model = Model(inputs=[iv_input, cov_input], outputs=output)

# optimizer = Adam(learning_rate=learning_rate, epsilon=epsilon)
# model.compile(optimizer=optimizer, loss='mse')

# model.summary()


In [128]:
# def make_dataset(iv_array):
#     iv_dataset = tf.keras.utils.timeseries_dataset_from_array(
#         data=iv_array[:,:,:,0:1],
#         targets=None,
#         sequence_length=window_size,
#         sequence_stride=1,
#         batch_size=batch_size,
#         shuffle=False
#     )

#     cov_dataset = tf.keras.utils.timeseries_dataset_from_array(
#     data=covariates_full,
#     targets=None,
#     sequence_length=window_size,
#     sequence_stride=1,
#     batch_size=batch_size,
#     shuffle=False
# )
    
#     target_dataset = tf.keras.utils.timeseries_dataset_from_array(
#     data=iv_array[window_size:,:,:,0:1],  # This aligns with windowed inputs
#     targets=None,
#     sequence_length=1,
#     sequence_stride=1,
#     batch_size=batch_size,
#     shuffle=False
# )
#     # Flatten target sequence dim (from (batch, 1, H, W, 1) → (batch, H, W, 1))
#     target_dataset = target_dataset.map(lambda x: tf.squeeze(x, axis=1))

#     final_dataset = tf.data.Dataset.zip(((iv_dataset, cov_dataset), target_dataset))

#     return final_dataset


In [129]:
# model.fit([iv_array, covariate_array], target_array)

In [130]:
TIME_STEPS = window_size
HEIGHT = len(data_train['moneyness_enc'].unique())
WIDTH = len(data_train['maturity'].unique())
CHANNELS = 1 +len(covariate_columns)

model = Sequential()

# ConvLSTM2D expects 5D input: (batch, time, height, width, channels)
model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
                     padding='same', return_sequences=True,
                     input_shape=(TIME_STEPS, HEIGHT, WIDTH, CHANNELS)))
model.add(BatchNormalization())

model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
                     padding='same', return_sequences=False))
model.add(BatchNormalization())

# Final 3D convolution to map to the next frame
model.add(tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 1),
                                 activation='sigmoid', padding='same'))

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon)
model.compile(loss='mse', optimizer=optimizer)

# Double check the architecture, and the activaiton function
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_8 (ConvLSTM2D)  (None, 21, 8, 5, 64)      150016    
                                                                 
 batch_normalization_8 (Batc  (None, 21, 8, 5, 64)     256       
 hNormalization)                                                 
                                                                 
 conv_lstm2d_9 (ConvLSTM2D)  (None, 8, 5, 64)          295168    
                                                                 
 batch_normalization_9 (Batc  (None, 8, 5, 64)         256       
 hNormalization)                                                 
                                                                 
 conv2d_4 (Conv2D)           (None, 8, 5, 1)           65        
                                                                 
Total params: 445,761
Trainable params: 445,505
Non-tr

In [131]:
# model = Sequential([
#     ConvLSTM2D(64, (3,3), padding='same', return_sequences=True,
#               input_shape=(TIME_STEPS, HEIGHT, WIDTH, CHANNELS)),
#     BatchNormalization(),
#     ConvLSTM2D(128, (3,3), padding='same', return_sequences=True),  # Increased filters
#     BatchNormalization(),
#     ConvLSTM2D(64, (3,3), padding='same', return_sequences=False),
#     BatchNormalization(),
#     Conv2D(1, (1,1), activation='sigmoid', padding='same')
# ])
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon)
# model.compile(loss='mse', optimizer=optimizer)


In [132]:
model.fit(dataset_train, epochs=epochs, validation_data=dataset_val, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')])

# Takes about 40 min on dell xps laptop, 6.5 min on PC for short term
# 24 min on PC for long term

Epoch 1/100
75/75 [==============================] - 10s 109ms/step - loss: 0.1115 - val_loss: 0.0586
Epoch 2/100
75/75 [==============================] - 8s 105ms/step - loss: 0.0425 - val_loss: 0.0516
Epoch 3/100
75/75 [==============================] - 8s 104ms/step - loss: 0.0376 - val_loss: 0.0525
Epoch 4/100
75/75 [==============================] - 8s 105ms/step - loss: 0.0353 - val_loss: 0.0674
Epoch 5/100
75/75 [==============================] - 8s 104ms/step - loss: 0.0173 - val_loss: 0.0605
Epoch 6/100
75/75 [==============================] - 8s 105ms/step - loss: 0.0136 - val_loss: 0.0549
Epoch 7/100
75/75 [==============================] - 8s 105ms/step - loss: 0.0126 - val_loss: 0.0478
Epoch 8/100
75/75 [==============================] - 8s 104ms/step - loss: 0.0121 - val_loss: 0.0390
Epoch 9/100
75/75 [==============================] - 8s 105ms/step - loss: 0.0116 - val_loss: 0.0354
Epoch 10/100
75/75 [==============================] - 8s 106ms/step - loss: 0.0114 - val_l

In [133]:
# Next step; functions for the IVRMSE and R_oos!!!!!
# H-step ahead performance!!!!!!
# WRITE IT ALL TO RESULTS!!!!!!
# Lower learning rate, beneficial, or leave it? Probably leave it
# All the HYPERPARAMETERS!!!!!! kernel strides, window sizes, parameters, layers, ALL OF THEM
# COVARIATES!!!!!!, Do all the hyperparameters again, and then only save the TEST PERFORMANCE
# MODEL ARCHITECTURE!!!!
# Investigate what happens, if you leave it as 0... We cannot do the interpolation properly, to be frank.

#THEN LONG TERM !!!!!!
# Transformer models!!!!!!!!!!!!!!!!
pred_val = model.predict(dataset_val)
pred_test = model.predict(dataset_test)

7/7 [==============================] - 0s 22ms/step


In [134]:
# Put it in the compile, but also call it afterward
# double check formula and also make it for R_oos
# Should also work per h-step ahead 
def calculate_ivrmse(y_true, y_pred, all_points=False):
    if not all_points:
        ivrmse = tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))
    else:
        sq_error = tf.square(y_true - y_pred)
        error_surface = tf.reduce_mean(sq_error, axis=[1 , 2])
        ivrmse = tf.sqrt(error_surface)

    return ivrmse.numpy()

# model.compile(optimizer='adam', loss='mse', metrics=[ivrmse_metric])


In [135]:
print(IV_val.shape, pred_val.shape, IV_test_input.shape, IV_test_input[:-window_size].shape)

(114, 8, 5, 1) (114, 8, 5, 1) (215, 8, 5, 1) (194, 8, 5, 1)


In [136]:
print(dataset_val)

<BatchDataset element_spec=(TensorSpec(shape=(None, None, 8, 5, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, 8, 5, 1), dtype=tf.float64, name=None))>


In [137]:
for x_batch, y_batch in dataset_val.take(1):
    print("Input batch shape:", x_batch.shape)
    print("Target batch shape:", y_batch.shape)

Input batch shape: (32, 21, 8, 5, 1)
Target batch shape: (32, 8, 5, 1)


In [138]:
print(calculate_ivrmse(IV_val, pred_val))
print(calculate_ivrmse(IV_val, pred_val, all_points=False))
# The loss is less, than in the optimization.. probably a different metric, or formula than mse
# check values of the papers that are like yours

0.19759575224583834
0.19759575224583834


In [139]:
def calculate_r_oos(y_true, y_pred, all_points=False):
    if not all_points:
        ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
        mean_IV = tf.reduce_mean(y_true, axis=[1, 2], keepdims=True) # should be shape of 114 long
        # print(mean_IV[0], mean_IV[1], mean_IV[2])
        # print(y_true[0], y_true[1], y_true[2])
        ss_tot = tf.reduce_sum(tf.square(y_true - mean_IV))
        # print((y_true - mean_IV)[0])
        r2 = 1 - ss_res/ss_tot
    else:
        ss_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=[1, 2])
        mean_IV = tf.reduce_mean(y_true, axis=[1, 2], keepdims=True)
        ss_tot = tf.reduce_sum(tf.square(y_true - mean_IV), axis=[1, 2])
        r2 = 1 - ss_res/ss_tot
    return r2.numpy()

print(calculate_r_oos(IV_val, pred_val))

0.12325646653694333


In [140]:
# Train model again on BOTH train and validation, and then investigate the TEST PERFORMANCE!!

In [141]:
print(calculate_ivrmse(IV_test, pred_test))
print(calculate_r_oos(IV_test, pred_test))

0.3162442861025512
-0.6135722871043572


In [142]:
def get_results(y_real, y_pred):
    ivrmse = calculate_ivrmse(y_real, y_pred)
    ivrmse_h = calculate_ivrmse(y_real, y_pred, all_points=True)
    r_oos = calculate_r_oos(y_real, y_pred)
    r_oos_h = calculate_r_oos(y_real, y_pred, all_points=True)

    return ivrmse, ivrmse_h, r_oos, r_oos_h

In [143]:
def write_results(folder_path, ivrmse, r_oos, ivrmse_h, r_oos_h, surface, surface_pred, covariate_columns, option_type):

    ivrmse_path = folder_path / Path("ivrmse")
    r_oos_path = folder_path / Path("r_oos")
    ivrmse_h_path = folder_path / Path("ivrmse_h")
    r_oos_h_path = folder_path / Path("r_oos_h")
    surface_path = folder_path / Path("surface")
    surface_pred_path = folder_path / Path("surface_pred")

    if not ivrmse_path.exists():
        ivrmse_path.mkdir(parents=True, exist_ok=True)

    if not r_oos_path.exists():
        r_oos_path.mkdir(parents=True, exist_ok=True)

    if not ivrmse_h_path.exists():
        ivrmse_h_path.mkdir(parents=True, exist_ok=True)

    if not r_oos_h_path.exists():
        r_oos_h_path.mkdir(parents=True, exist_ok=True)

    if not surface_path.exists():
        surface_path.mkdir(parents=True, exist_ok=True)

    if not surface_pred_path.exists():
        surface_pred_path.mkdir(parents=True, exist_ok=True)

    cov = ""
    for i in covariate_columns:
        cov = cov+ "_" +i
    np.save(ivrmse_path / f"{option_type}_ws_{window_size}_h_{h_step}{cov}.npy", ivrmse)
    np.save(r_oos_path / f"{option_type}_ws_{window_size}_h_{h_step}{cov}.npy", r_oos)
    np.save(ivrmse_h_path / f"{option_type}_ws_{window_size}_h_{h_step}{cov}.npy", ivrmse_h)
    np.save(r_oos_h_path / f"{option_type}_ws_{window_size}_h_{h_step}{cov}.npy", r_oos_h)
    np.save(surface_path/ f"{option_type}_ws_{window_size}_h_{h_step}{cov}.npy", surface)
    np.save(surface_pred_path / f"{option_type}_ws_{window_size}_h_{h_step}{cov}.npy", surface_pred)

In [144]:
folder_path = Path(f"results/test_{run}")
ivrmse, ivrmse_h, r_oos, r_oos_h = get_results(IV_test, pred_test)
write_results(folder_path, ivrmse, r_oos, ivrmse_h, r_oos_h, IV_test, pred_test, covariate_columns, option_type)

In [145]:
print(r_oos_h)

[[-1.79138179]
 [-0.054546  ]
 [-0.07861797]
 [-0.02036807]
 [-0.9430473 ]
 [ 0.49018698]
 [-2.89775558]
 [-5.56639987]
 [ 0.33615971]
 [-0.04225509]
 [-0.74754414]
 [-2.6001873 ]
 [-0.54973219]
 [-2.22626067]
 [-0.29799605]
 [-1.32107186]
 [-1.19425315]
 [-0.3349431 ]
 [-1.64188281]
 [-3.38089759]
 [-0.64835454]
 [-5.72020788]
 [-2.154226  ]
 [-0.77499511]
 [-0.42104429]
 [-3.48672618]
 [-1.23533048]
 [-0.15511529]
 [ 0.32572071]
 [-0.02194204]
 [ 0.28163691]
 [-2.98577342]
 [-1.57563865]
 [-0.51802128]
 [-3.70804855]
 [-1.6062541 ]
 [-2.66563657]
 [ 0.28836503]
 [-0.44486318]
 [-0.13858459]
 [-0.72705505]
 [-0.86995016]
 [-0.8239676 ]
 [-0.3264601 ]
 [-0.10977858]
 [-2.52352308]
 [-0.12486719]
 [-0.64200276]
 [-1.75270436]
 [-0.86945937]
 [-1.59915758]
 [ 0.09784615]
 [-0.56155372]
 [-1.65522483]
 [-0.25272191]
 [-0.38040472]
 [-0.1545394 ]
 [-0.72907413]
 [-0.89032976]
 [-4.70057795]
 [ 0.1430873 ]
 [ 0.36449217]
 [-0.04719884]
 [-0.76839018]
 [-1.22367856]
 [-2.52587192]
 [ 0.24775

In [146]:
folder_path = Path(f"results/validation_{run}")
ivrmse, ivrmse_h, r_oos, r_oos_h = get_results(IV_val, pred_val)
write_results(folder_path, ivrmse, r_oos, ivrmse_h, r_oos_h, IV_test, pred_test, covariate_columns, option_type)

In [147]:
# For the h-step ahead forecast, an autoregressive approach is used
# Predict the one step ahead forecast (done now)
# After, use this one step ahead forecast in the predict command. 

In [148]:
print(pred_test.shape)

(194, 8, 5, 1)
